In [ ]:
import json
from tiny_ball_detector.models import TrackerNet
import keras

from logging import DEBUG
import logging
from pathlib import Path
from tiny_ball_detector.dataset.tracknet_dataset import TrackNetDataset
import tensorflow as tf
import os


def resize(image, label):
    return (
        keras.preprocessing.image.smart_resize(image, (360, 640)),
        keras.preprocessing.image.smart_resize(label, (360, 640)),
    )


logger = logging.getLogger("FrameGeneratorLogger")

trackNetDataset = TrackNetDataset(
    path=Path("../data/tennis"), n_frames=3, logger=logger
)

dataset = (
    trackNetDataset.dataset.batch(3)
    .map(resize, num_parallel_calls=tf.data.AUTOTUNE)
    .prefetch(tf.data.AUTOTUNE)
    .cache()
)

In [ ]:
tracknet_model = TrackerNet()

tracknet_model.build((None, 360, 640, 9))

tracknet_model.summary()

In [ ]:
# Définir un chemin pour enregistrer les checkpoints
checkpoint_path = "checkpoints/tracker_net.weights.h5"

# Callback pour sauvegarder les poids du modèle
checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,  # Sauvegarde uniquement les poids (pas l'architecture)
    save_best_only=True,  # Sauvegarde uniquement le meilleur modèle
    monitor="val_loss",  # Basé sur la validation loss
    mode="min",  # Sauvegarde lorsque la loss diminue
    verbose=1,
)

# Compilation du modèle
tracknet_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)


tracknet_model.fit(
    dataset,
    epochs=10,
    callbacks=[checkpoint_callback],
)

Epoch 1/10


W0000 00:00:1740940460.200395   28166 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert


2193/2194 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step - accuracy: 0.5445 - loss: 5.4584

W0000 00:00:1740940927.799857   28167 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert


2194/2194 ━━━━━━━━━━━━━━━━━━━━ 482s 215ms/step - accuracy: 0.5447 - loss: 5.4583
Epoch 2/10


/home/gaetan/.cache/pypoetry/virtualenvs/tiny-ball-detector-LtrcJND9-py3.12/lib/python3.12/site-packages/keras/src/callbacks/model_checkpoint.py:209: UserWarning: Can save best model only with val_loss available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)


  58/2194 ━━━━━━━━━━━━━━━━━━━━ 7:32 212ms/step - accuracy: 0.9996 - loss: 5.1059

KeyboardInterrupt: 

In [ ]:
import json
from tiny_ball_detector.models import TrackerNet
import keras

from logging import DEBUG
import logging
from pathlib import Path
from tiny_ball_detector.dataset.tracknet_dataset import TrackNetDataset
import tensorflow as tf
import os


print("GPUs disponibles :", tf.config.list_physical_devices("GPU"))
BATCH_SIZE = 9

strategy = strategy = tf.distribute.MirroredStrategy()


logger = logging.getLogger("FrameGeneratorLogger")

trackNetDataset = TrackNetDataset(
    path=Path("../data/tennis"), n_frames=3, logger=logger
)


with strategy.scope():

    tracknet_model = TrackerNet()

    tracknet_model.build((None, 360, 640, 9))

    # Définir un chemin pour enregistrer les checkpoints
    checkpoint_path = "checkpoints/tracker_net.weights.h5"

    # Callback pour sauvegarder les poids du modèle
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_path,
        save_weights_only=True,  # Sauvegarde uniquement les poids (pas l'architecture)
        save_best_only=True,  # Sauvegarde uniquement le meilleur modèle
        monitor="val_loss",  # Basé sur la validation loss
        mode="min",  # Sauvegarde lorsque la loss diminue
        verbose=1,
    )

    # Compilation du modèle
    tracknet_model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-4),
        loss=keras.losses.SparseCategoricalCrossentropy(),
        metrics=["precision", "recall", "f1_score"],
    )

    dataset = trackNetDataset.dataset(BATCH_SIZE, (360, 640))

tracknet_model.fit(
    dataset,
    epochs=10,
    callbacks=[checkpoint_callback],
)

GPUs disponibles : [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU')]
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2')


2025-03-02 18:52:24.315605: W tensorflow/core/kernels/data/cache_dataset_ops.cc:914] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2025-03-02 18:52:31.128404: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 82944000 bytes after encountering the first element of size 82944000 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
Epoch 1/10


2025-03-02 18:52:33.212010: W tensorflow/core/kernels/data/cache_dataset_ops.cc:914] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


INFO:tensorflow:Collective all_reduce tensors: 1 all_reduces, num_devices = 3, group_size = 3, implementation = CommunicationImplementation.NCCL, num_packs = 1
INFO:tensorflow:Collective all_reduce tensors: 1 all_reduces, num_devices = 3, group_size = 3, implementation = CommunicationImplementation.NCCL, num_packs = 1
INFO:tensorflow:Collective all_reduce tensors: 72 all_reduces, num_devices = 3, group_size = 3, implementation = CommunicationImplementation.NCCL, num_packs = 1
INFO:tensorflow:Collective all_reduce tensors: 1 all_reduces, num_devices = 3, group_size = 3, implementation = CommunicationImplementation.NCCL, num_packs = 1
INFO:tensorflow:Collective all_reduce tensors: 1 all_reduces, num_devices = 3, group_size = 3, implementation = CommunicationImplementation.NCCL, num_packs = 1


2025-03-02 18:52:43.369358: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 82944000 bytes after encountering the first element of size 82944000 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


 94/732 ━━━━━━━━━━━━━━━━━━━━ 7:05 667ms/step - accuracy: nan - loss: nan

KeyboardInterrupt: 

In [5]:
def create_compiled_model():
    tracknet_model = TrackerNet()

    tracknet_model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-4),
        loss=keras.losses.SparseCategoricalCrossentropy(),
        metrics=["precision", "recall", "f1_score"],
    )

In [ ]:
import time


gpus = tf.config.list_physical_devices("GPU")
print(f"GPUs disponibles : {gpus}")

BATCH_SIZE = len(gpus) * 3
EPOCHS = 1


def benchmark_single_gpu(model, dataset):
    start_time = time.time()

    time_1gpu = time.time() - start_time